# NBA BetIQ – Expected Value (EV) Simulation

This notebook uses model probabilities and sportsbook odds to compute
expected value (EV) and simulate betting strategies over time.


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ml.visuals import plot_ev_over_time

sns.set(style="whitegrid")


## 1. Load Model Predictions and Betting Lines

We assume we have a dataset with columns:
- `date`
- `game_id`
- `team`
- `moneyline` (American odds)
- `model_prob` (predicted win probability)
- `actual_outcome` (1 = win, 0 = loss)


In [ ]:
preds_path = "../data/processed/model_predictions.csv"
df = pd.read_csv(preds_path, parse_dates=["date"])

df.head()


## 2. Helper Functions – EV Calculation


In [ ]:
def american_odds_to_multiplier(odds):
    """
    Convert American odds to net payout multiplier.

    Example:
        +150 -> 1.5  (risk 1, win 1.5)
        -120 -> 0.833... (risk 1, win 0.833...)
    """
    odds = float(odds)
    if odds > 0:
        return odds / 100.0
    else:
        return 100.0 / abs(odds)


def compute_ev_row(prob, odds, stake=1.0):
    """
    Compute expected value of a single bet.

    EV = p(win) * profit_if_win + p(loss) * profit_if_loss
       = p(win) * (multiplier * stake) + (1 - p(win)) * (-stake)
    """
    mult = american_odds_to_multiplier(odds)
    ev = prob * (mult * stake) + (1 - prob) * (-stake)
    return ev


## 3. Compute EV for Each Bet


In [ ]:
df["expected_value"] = df.apply(
    lambda row: compute_ev_row(row["model_prob"], row["moneyline"], stake=1.0),
    axis=1
)

df[["model_prob", "moneyline", "expected_value"]].head()


## 4. Define a Betting Strategy

For example: bet when EV ≥ 0.02 (2% edge) and model probability
is between 0.40 and 0.65 to avoid extreme tails.


In [ ]:
edge_threshold = 0.02
min_prob = 0.40
max_prob = 0.65

df["bet_flag"] = (
    (df["expected_value"] >= edge_threshold)
    & (df["model_prob"].between(min_prob, max_prob))
).astype(int)

df["bet_flag"].value_counts()


## 5. Simulate Results (Units Won/Lost)


In [ ]:
stake = 1.0

def realized_profit(row, stake=1.0):
    if row["bet_flag"] == 0:
        return 0.0
    mult = american_odds_to_multiplier(row["moneyline"])
    if row["actual_outcome"] == 1:
        return mult * stake
    else:
        return -stake

df["profit"] = df.apply(realized_profit, axis=1)
df["cumulative_profit"] = df["profit"].cumsum()

df[["date", "bet_flag", "profit", "cumulative_profit"]].tail()


## 6. Visualize Cumulative EV vs Realized Profit


In [ ]:
# Aggregate by date
daily = df.groupby("date").agg(
    daily_ev=("expected_value", "sum"),
    daily_profit=("profit", "sum"),
    num_bets=("bet_flag", "sum"),
).reset_index()

daily["cum_ev"] = daily["daily_ev"].cumsum()
daily["cum_profit"] = daily["daily_profit"].cumsum()

plt.figure(figsize=(10, 6))
plt.plot(daily["date"], daily["cum_ev"], label="Cumulative EV")
plt.plot(daily["date"], daily["cum_profit"], label="Cumulative Profit")
plt.xlabel("Date")
plt.ylabel("Units")
plt.title("Cumulative EV vs Realized Profit Over Time")
plt.legend()
plt.tight_layout()
plt.show()


## 7. Save EV Plot via Helper

We can also store a standard EV-over-time plot in `/visuals`.


In [ ]:
plot_ev_over_time(
    df=df[df["bet_flag"] == 1],
    date_col="date",
    ev_col="expected_value",
    save_path="../visuals/ev_over_time_strategy.png",
)


## 8. Summary

We computed per-bet EV, defined a betting strategy, ran a simulation,
and compared theoretical EV to realized profit over time.
